In [ ]:
#ラベリングによる学習/検証データの準備

from PIL import Image
import os, glob
import numpy as np
import random, math

In [2]:
#画像が保存されているルートディレクトリのパス
root_dir = "tea_datasets"
# 商品名
categories = ["綾鷹","お〜いお茶","伊右衛門　贅沢冷茶","爽健美茶",
              "綾鷹　茶葉のあまみ","お〜いお茶　ほうじ茶","伊右衛門","お〜いお茶　濃い茶",
              "生茶","十六茶"]

# 画像データ用配列
X = []
# ラベルデータ用配列
Y = []



In [3]:
#画像データごとにadd_sample()を呼び出し、X,Yの配列を返す関数
def make_sample(files):
    global X, Y
    X = []
    Y = []
    for cat, fname in files:
        add_sample(cat, fname)
    return np.array(X), np.array(Y)

#渡された画像データを読み込んでXに格納し、また、
#画像データに対応するcategoriesのidxをY格納する関数
def add_sample(cat, fname):
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((150, 150))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

#全データ格納用配列
allfiles = []

#カテゴリ配列の各値と、それに対応するidxを認識し、全データをallfilesにまとめる
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + '/*.jpg')#*は何でもいいけど、.jpgのファイルのすべてを対象にとっている
    for f in files:
        allfiles.append((idx, f))
        
        
    
    

In [ ]:
#シャッフル後、学習データと検証データに分ける
random.shuffle(allfiles)
th = math.floor(len(allfiles) * 0.8)
train = allfiles[0:th]
test  = allfiles[th:]
X_train, y_train = make_sample(train)
X_test, y_test = make_sample(test)
xy = (X_train, X_test, y_train, y_test)
#データを保存する（データの名前を「tea_data.npy」としている）
np.save("/Users/arimachishun/Desktop/ayataka/tea_data.npy", xy)


In [ ]:
#モデルの構築

from keras import layers, models

model = models.Sequential()
model.add(layers.Conv2D(32,(3,3),activation="relu",input_shape=(150,150,3)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128,(3,3),activation="relu"))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(512,activation="relu"))
model.add(layers.Dense(10,activation="sigmoid")) #分類先の種類分設定

#モデル構成の確認
model.summary()

In [ ]:
#モデルのコンパイル

from keras import optimizers

model.compile(loss="binary_crossentropy",
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=["acc"])

In [ ]:
#データの準備

from keras.utils import np_utils
import numpy as np

categories = ["綾鷹","お〜いお茶","伊右衛門　贅沢冷茶","爽健美茶",
              "綾鷹　茶葉のあまみ","お〜いお茶　ほうじ茶","伊右衛門","お〜いお茶　濃い茶",
              "生茶","十六茶"]
nb_classes = len(categories)

X_train, X_test, y_train, y_test = np.load("/Users/arimachishun/Desktop/ayataka/tea_data.npy")

#データの正規化
X_train = X_train.astype("float") / 255
X_test  = X_test.astype("float")  / 255

#kerasで扱えるようにcategoriesをベクトルに変換
y_train = np_utils.to_categorical(y_train, nb_classes)
y_test  = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
#モデルの学習

model = model.fit(X_train,
                  y_train,
                  epochs=10,
                  batch_size=6,
                  validation_data=(X_test,y_test))

In [ ]:
#学習結果を表示

import matplotlib.pyplot as plt

acc = model.history['acc']
val_acc = model.history['val_acc']
loss = model.history['loss']
val_loss = model.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('精度を示すグラフのファイル名')

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('損失値を示すグラフのファイル名')

In [ ]:
#モデルの保存

json_string = model.model.to_json()
open('/Users/arimachishun/Desktop/ayataka/tea_predict.json', 'w').write(json_string)

#重みの保存

hdf5_file = "/Users/arimachishun/Desktop/ayataka/tea_predict.hdf5"
model.model.save_weights(hdf5_file)

In [ ]:
from PIL import Image
import os, glob
import numpy as np
import random, math

# 画像が保存されているディレクトリのパス
root_dir = "tea_datasets"
# 画像が保存されているフォルダ名
categories = ["綾鷹","お〜いお茶","伊右衛門　贅沢冷茶","爽健美茶",
              "綾鷹　茶葉のあまみ","お〜いお茶　ほうじ茶","伊右衛門","お〜いお茶　濃い茶",
              "生茶","十六茶"]

X = [] # 画像データ
Y = [] # ラベルデータ

# フォルダごとに分けられたファイルを収集
#（categoriesのidxと、画像のファイルパスが紐づいたリストを生成）
allfiles = []
for idx, cat in enumerate(categories):
    image_dir = root_dir + "/" + cat
    files = glob.glob(image_dir + "/*.jpg")
    for f in files:
        allfiles.append((idx, f))

for cat, fname in allfiles:
    img = Image.open(fname)
    img = img.convert("RGB")
    img = img.resize((150, 150))
    data = np.asarray(img)
    X.append(data)
    Y.append(cat)

x = np.array(X)
y = np.array(Y)

np.save("tea_data_test_X/tea_data_test_X_150.npy", x)
np.save("tea_data_test_Y/tea_data_test_Y_150.npy", y)

In [ ]:
# モデルの精度を測る

#評価用のデータの読み込み
test_X = np.load("tea_data_test_X/tea_data_test_X_150.npy")
test_Y = np.load("tea_data_test_Y/tea_data_test_Y_150.npy")

#Yのデータをone-hotに変換
from keras.utils import np_utils

test_Y = np_utils.to_categorical(test_Y, 10)

score = model.model.evaluate(x=test_X,y=test_Y)

print('loss=', score[0])
print('accuracy=', score[1])

In [ ]:
#綾鷹を選ばせるプログラム

from keras import models
from keras.models import model_from_json
from keras.preprocessing import image
import numpy as np

#保存したモデルの読み込み
model = model_from_json(open('/Users/arimachishun/Desktop/ayataka/tea_predict.json').read())
#保存した重みの読み込み
model.load_weights('/Users/arimachishun/Desktop/ayataka/tea_predict.hdf5')

categories = ["綾鷹","お〜いお茶","伊右衛門　贅沢冷茶","爽健美茶",
              "綾鷹　茶葉のあまみ","お〜いお茶　ほうじ茶","伊右衛門","お〜いお茶　濃い茶",
              "生茶","十六茶"]

#画像を読み込む
img_path = str(input())
img = image.load_img(img_path,target_size=(150, 150, 3))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)

#予測
features = model.predict(x)

#予測結果によって処理を分ける
if features[0,0] == 1:
    print ("選ばれたのは、綾鷹でした。")

elif features[0,4] == 1:
    print ("選ばれたのは、綾鷹（茶葉のあまみ）でした。")

else:
    for i in range(0,10):
          if features[0,i] == 1:
              cat = categories[i]
    message = "綾鷹を選んでください。（もしかして：あなたが選んでいるのは「" + cat + "」ではありませんか？）"
    print(message)
